In [1]:
import numpy as np
import gym # pip intall gym
import random

In [2]:
env = gym.make('FrozenLake-v1', is_slippery = False, new_step_api=True, render_mode='ansi')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# 4 action to perform with 1 step decision
env.action_space

Discrete(4)

In [4]:
env.observation_space

Discrete(16)

In [5]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

# Create initial Q table
qtable = np.zeros((state_space_size, action_space_size))
print("Q Table\n",qtable)

Q Table
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [41]:
total_episodes = 15000
learning_rate = 0.2
max_steps = 100
gamma = 0.99

epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
decay_rate = 0.001

In [42]:
def converged_qtable(lst_qtables, interval=100, convergence_threshold=0.1):
    if len(lst_qtables) < interval:
        return False  # Not enough episodes to compare

    avg_change = (lst_qtables[-1] - np.mean(lst_qtables[-interval:-1])) / interval

    return True

In [43]:
# Traning the agent
rewards = []
lst_qtables = []
num_optimal_episode = 0

for episode in range(total_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    for step in range(max_steps):

        if random.uniform(0,1) > epsilon:
            action = np.argmax(qtable[state,:]) # Exploit
        else:
            # This is same thing as np.random.choice(action)
            action = env.action_space.sample() # Explore

        # terminated is a step that reach to a positive or negative done state.
        # truncated is a step that reach to out of bounds or strategy or rules.
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        max_new_state = np.max(qtable[new_state,:])

        qtable[state,action] = qtable[state,action] + learning_rate*(reward+gamma*max_new_state-qtable[state,action])
        lst_qtables.append(qtable[state,action])
        total_rewards += reward

        state = new_state
        if done:
            break

    num_optimal_episode += 1
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)

    # Check converge  of q-table when equal to required score
    if  sum(rewards)/total_episodes > 0.9:
      if converged_qtable(lst_qtables):
        break

# Max score = 1 or 100%
print("Score:", str(sum(rewards)/total_episodes))
print(f"lasted update of qtable: \n{qtable}")
print(f"Total setup training episode {total_episodes}")
print(f"Total optimal training episode {num_optimal_episode}")

Score: 0.9000666666666667
lasted update of qtable: 
[[0.94148015 0.95099005 0.95099005 0.94148015]
 [0.94148015 0.         0.96059601 0.95099005]
 [0.95099005 0.970299   0.95099005 0.96059601]
 [0.96059601 0.         0.95099005 0.95099005]
 [0.95099005 0.96059601 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.         0.96059601]
 [0.         0.         0.         0.        ]
 [0.96059601 0.         0.970299   0.95099005]
 [0.96059601 0.9801     0.9801     0.        ]
 [0.970299   0.99       0.         0.970299  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.99       0.970299  ]
 [0.9801     0.99       1.         0.9801    ]
 [0.         0.         0.         0.        ]]
Total setup training episode 15000
Total optimal training episode 14726


In [44]:
# Let's agent play n times
env.reset()

n = 10
for episode in range(n):
    state = env.reset()
    step = 0
    done = False

    print("Episode:", episode+1)

    for step in range(max_steps):
        action = np.argmax(qtable[state,:])
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        if done:
            print(env.render())
            print("Number of Steps:", step)
            break
        state = new_state

env.close()


Episode: 1
['\n\x1bS\x1bFFF\nFHFH\nFFFH\nHFFG\n', '  (Down)\nSFFF\n\x1bF\x1bHFH\nFFFH\nHFFG\n', '  (Down)\nSFFF\nFHFH\n\x1bF\x1bFFH\nHFFG\n', '  (Right)\nSFFF\nFHFH\nF\x1bF\x1bFH\nHFFG\n', '  (Down)\nSFFF\nFHFH\nFFFH\nH\x1bF\x1bFG\n', '  (Right)\nSFFF\nFHFH\nFFFH\nHF\x1bF\x1bG\n', '  (Right)\nSFFF\nFHFH\nFFFH\nHFF\x1bG\x1b\n']
Number of Steps: 5
Episode: 2
['\n\x1bS\x1bFFF\nFHFH\nFFFH\nHFFG\n', '  (Down)\nSFFF\n\x1bF\x1bHFH\nFFFH\nHFFG\n', '  (Down)\nSFFF\nFHFH\n\x1bF\x1bFFH\nHFFG\n', '  (Right)\nSFFF\nFHFH\nF\x1bF\x1bFH\nHFFG\n', '  (Down)\nSFFF\nFHFH\nFFFH\nH\x1bF\x1bFG\n', '  (Right)\nSFFF\nFHFH\nFFFH\nHF\x1bF\x1bG\n', '  (Right)\nSFFF\nFHFH\nFFFH\nHFF\x1bG\x1b\n']
Number of Steps: 5
Episode: 3
['\n\x1bS\x1bFFF\nFHFH\nFFFH\nHFFG\n', '  (Down)\nSFFF\n\x1bF\x1bHFH\nFFFH\nHFFG\n', '  (Down)\nSFFF\nFHFH\n\x1bF\x1bFFH\nHFFG\n', '  (Right)\nSFFF\nFHFH\nF\x1bF\x1bFH\nHFFG\n', '  (Down)\nSFFF\nFHFH\nFFFH\nH\x1bF\x1bFG\n', '  (Right)\nSFFF\nFHFH\nFFFH\nHF\x1bF\x1bG\n', '  (Right)\nSFFF\nFHFH